In [2]:
%cd ~/dv/sns-chops/resolution/ARCS/
import os, numpy as np

/SNS/users/lj7/dv/sns-chops/resolution/ARCS


In [3]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py

In [4]:
import plotly

In [5]:
import plotly.figure_factory as ff

In [32]:
import resolution_plot
reload(resolution_plot)

<module 'resolution_plot' from 'resolution_plot.py'>

In [33]:
fc1data = resolution_plot.ExpData('./V_Cali_Int_Res_FC1_2018_v2.dat')

reading data. please wait...
  done


In [34]:
fc2data = resolution_plot.ExpData('./V_Cali_Int_Res_FC2_2018_v2.dat')

reading data. please wait...
  done


In [35]:
fc1_highres_data = resolution_plot.ExpData('./V_Cali_Int_Res_FC1_HighRes_2018_v2.dat')

reading data. please wait...
  done


In [36]:
unique_nominal_Eis = set( list(fc1data.Ei_list) + list(fc2data.Ei_list) + list(fc1_highres_data.Ei_list) )

In [37]:
unique_nominal_Eis = sorted(list(unique_nominal_Eis))

# Intensity vs FWHM

In [38]:
Eis = unique_nominal_Eis #[:10]
title='ARCS flux (intensity) vs resolution Version 2 \nuse dropdown list from left to choose incident energy'
data =[]
buttons = []
all_hidden = [False] * len(Eis) * 3 # FC 1 and 2 and FC 1 HighRes
extra_info = dict(
    chopper_freqs = ('nu', '%sHz')
)
for i, n_Ei in enumerate(Eis):
    plot = fc1data.createPlotXY(n_Ei, 'FWHM', 'intensity', extra_info=extra_info)
    plot.visible = False
    plot.name = 'chopper1'
    data.append(plot)
    #
    plot = fc2data.createPlotXY(n_Ei, 'FWHM', 'intensity', extra_info=extra_info)
    plot.visible = False
    plot.name = 'chopper2'
    data.append(plot)
    # 
    plot = fc1_highres_data.createPlotXY(n_Ei, 'FWHM', 'intensity', extra_info=extra_info)
    plot.visible = False
    plot.name = 'chopper1-HighRes'
    data.append(plot)
    #
    visible = list(all_hidden)
    visible[3*i] = True; visible[3*i+1] = True; visible[3*i+2] = True
    #
    button =  dict(
        label = '%smeV' % n_Ei,
        method = 'update',
        args = [{'visible': visible},
                {'title': 'Ei=%smeV %s' % (n_Ei, title),
                }])
    buttons.append(button)
    continue

updatemenus = list([
    dict(active=-1,
         buttons=list(buttons)
        )
])

atEi = Eis.index(100.)
data[atEi*3].visible = data[atEi*3+1].visible = data[atEi*3+2].visible = True

layout = dict(
    title=title, 
    # showlegend=False, 
    updatemenus=updatemenus,
    xaxis=dict(
        title='FWHM (meV)',
    ),
    yaxis=dict(
        title='Flux -- Peak area (arb. unit)',
    )
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='ARCS flux (intensity) vs resolution: Version 2')

# Peak height vs FWHM

In [39]:
Eis = unique_nominal_Eis #[:10]
title='ARCS flux (peak height) vs resolution Version 2 (use dropdown list from left to choose incident energy)'
data =[]
buttons = []
all_hidden = [False] * len(Eis) * 3 # FC 1 and 2 and FC1-HighRes
extra_info = dict(
    chopper_freqs = ('nu', '%sHz')
)
for i, n_Ei in enumerate(Eis):
    plot = fc1data.createPlotXY(n_Ei, 'FWHM', 'Height', extra_info=extra_info)
    plot.visible = False
    plot.name = 'chopper1'
    data.append(plot)
    #
    plot = fc2data.createPlotXY(n_Ei, 'FWHM', 'Height', extra_info=extra_info)
    plot.visible = False
    plot.name = 'chopper2'
    data.append(plot)
    #
    plot = fc1_highres_data.createPlotXY(n_Ei, 'FWHM', 'Height', extra_info=extra_info)
    plot.visible = False
    plot.name = 'chopper1-HighRes'
    data.append(plot)
    # 
    visible = list(all_hidden)
    visible[3*i] = True; visible[3*i+1] = True; visible[3*i+2] = True
    #
    button =  dict(
        label = '%smeV' % n_Ei,
        method = 'update',
        args = [{'visible': visible},
                {'title': 'Ei=%smeV %s' % (n_Ei, title),
                }])
    buttons.append(button)
    continue

updatemenus = list([
    dict(active=-1,
         buttons=list(buttons)
        )
])

atEi = Eis.index(100.)
data[atEi*3].visible = data[atEi*3+1].visible = data[atEi*3+2].visible = True

layout = dict(
    title=title, 
    # showlegend=False, 
    updatemenus=updatemenus,
    xaxis=dict(
        title='FWHM (meV)',
    ),
    yaxis=dict(
        title='Flux -- Peak height (arb. unit)',
    )
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='ARCS flux (peak height) vs resolution: Version 2')